In [1]:
import time
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [2]:
#q8
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 16962740224 (15.80 GB)
DFS Remaining: 16696569856 (15.55 GB)
DFS Used: 266170368 (253.84 MB)
DFS Used%: 1.57%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 83
	Missing blocks (with replication factor 1): 83
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.21.0.5:9866 (project-4-batch04-dn-1.project-4-batch04_default)
Hostname: 27b3cf235b47
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 266170368 (253.84 MB)
Non DFS Used: 8841535488 (8.23 GB)
DFS Remaining: 16696569856 (15.55 GB)
DFS Used%: 1.03%
DFS Rem

In [3]:
time.sleep(30)

In [4]:
#q9
r = requests.get("http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS")
r.raise_for_status()
single_status = r.json()

offset = 0
length = single_status['FileStatus']['length']
blockSize = single_status['FileStatus']['blockSize']

blockDict = {}

while offset < length:
    r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&noredirect=true")
    if r.status_code == 403:
        if "lost" in blockDict:
            blockDict["lost"] += 1
        else:
            blockDict["lost"] = 1
    else:
        id = r.json().get("Location").split(":")[1][2:]
        if id in blockDict:
            blockDict[id] += 1
        else:
            blockDict[id] = 1
    offset += blockSize
    
blockDict

{'27b3cf235b47': 84, 'lost': 83}

In [12]:
#q10
offset = 0
count = 0
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
f = hdfs.open_input_file("/single.csv")

while offset < length:
    r = requests.get(f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={offset}&noredirect=true")
    if r.status_code == 403:
        pass
    else:
        text = f.read_at(blockSize, offset).decode('utf-8')
        num_fam = text.count("Single Family")
        count += num_fam
        
    offset += blockSize
    
count

FSDataInputStream#close error:
IOException: Filesystem closedjava.io.IOException: Filesystem closed
	at org.apache.hadoop.hdfs.DFSClient.checkOpen(DFSClient.java:494)
	at org.apache.hadoop.hdfs.DFSInputStream.close(DFSInputStream.java:728)
	at java.base/java.io.FilterInputStream.close(FilterInputStream.java:180)
/arrow/cpp/src/arrow/status.cc:155: Failed to close HdfsReadableFile: IOError: HDFS CloseFile failed. Detail: [errno 255] Unknown error 255


223993